In [1]:
#### DO NOT RUN THIS CODE IN THE NOTEBOOK - copy paste into terminal in the HOME directory!

# git clone https://github.com/tensorflow/models.git

In [2]:
# !pip install s3fs -q
# !pip install tensorflow_hub -q
# !pip install tensorflow_datasets -q
# !pip install pycocotools -q
# !pip install gin-config -q
# !pip install immutabledict -q
# !pip install sentencepiece -q
# !pip install transformers -q
# !pip install -r official/projects/movinet/requirements.txt -q

In [1]:
import configparser

config = configparser.ConfigParser()
config.read('/home/ec2-user/.aws/credentials')

['/home/ec2-user/.aws/credentials']

In [2]:
# AWS credentials and S3 settings
aws_access_key_id = config["default"]['aws_access_key_id']
aws_secret_access_key = config["default"]['aws_secret_access_key']
bucket_name = 'asl-capstone'
s3_URI = 's3://asl-capstone/'

In [3]:
import s3fs
fs = s3fs.S3FileSystem(key=aws_access_key_id, secret=aws_secret_access_key)

import boto3
s3 = boto3.client('s3',aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key,
                  region_name = 'us-west-2')


In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

from sklearn.model_selection import train_test_split

tf.keras.backend.clear_session()

2023-11-16 01:59:10.345206: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-16 01:59:11.058696: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
# !export XLA_PYTHON_CLIENT_PREALLOCATE=false
# !export XLA_FLAGS="--xla_gpu_strict_conv_algorithm_picker=false --xla_gpu_force_compilation_parallelism=1"

In [8]:
# Load model directly
from transformers import AutoTokenizer, TFT5ForConditionalGeneration #, TFT5EncoderModel
# from transformers.modeling_tf_outputs import TFBaseModelOutputWithPastAndCrossAttentions

# with tf.device("/GPU:0"):
tokenizer = AutoTokenizer.from_pretrained("google/t5-v1_1-large")
model = TFT5ForConditionalGeneration.from_pretrained("google/t5-v1_1-large")
    # encoder_model = T5EncoderModel.from_pretrained("google/t5-v1_1-large")

model.trainable = False

# tokenizer = AutoTokenizer.from_pretrained("t5-large")
# model = TFT5ForConditionalGeneration.from_pretrained("t5-large")
# encoder_model = T5EncoderModel.from_pretrained("t5-large")



You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
2023-11-15 21:45:08.940976: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-15 21:45:08.961956: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUM

In [31]:
for i in range(4):
    model.encoder.block[i].trainable = True

In [39]:
video_files = pd.read_parquet("s3://asl-capstone/youtube-asl/test_sample/numpy_files/RGB/masterfile.parquet")
video_files = video_files.drop_duplicates().reset_index(drop=True)
video_files['movinet_path'] = video_files.path.str.replace("/RGB/", '/movinet_output/')

print(video_files.shape)
video_files.head(10)

(484, 3)


,caption,path,movinet_path
0,Welcome to the third round,s3://asl-capstone/youtube-asl/test_sample/nump...,s3://asl-capstone/youtube-asl/test_sample/nump...
1,of the Pearls announcement.,s3://asl-capstone/youtube-asl/test_sample/nump...,s3://asl-capstone/youtube-asl/test_sample/nump...
2,Today's category is called the,s3://asl-capstone/youtube-asl/test_sample/nump...,s3://asl-capstone/youtube-asl/test_sample/nump...
3,Hidden Pearls. What is it?,s3://asl-capstone/youtube-asl/test_sample/nump...,s3://asl-capstone/youtube-asl/test_sample/nump...
4,"Hidden Pearls...in other words,",s3://asl-capstone/youtube-asl/test_sample/nump...,s3://asl-capstone/youtube-asl/test_sample/nump...
5,they are known as the dark horses.,s3://asl-capstone/youtube-asl/test_sample/nump...,s3://asl-capstone/youtube-asl/test_sample/nump...
6,"Every now and then, they are seldom recognized",s3://asl-capstone/youtube-asl/test_sample/nump...,s3://asl-capstone/youtube-asl/test_sample/nump...
7,for their work or performance.,s3://asl-capstone/youtube-asl/test_sample/nump...,s3://asl-capstone/youtube-asl/test_sample/nump...
8,"Additionally, they are behind a big leader",s3://asl-capstone/youtube-asl/test_sample/nump...,s3://asl-capstone/youtube-asl/test_sample/nump...
9,"If not for them, the big leader wouldn't be here.",s3://asl-capstone/youtube-asl/test_sample/nump...,s3://asl-capstone/youtube-asl/test_sample/nump...


In [40]:
train_files, test_files = train_test_split(video_files, train_size=0.7)

print(train_files.shape)
print(test_files.shape)

(338, 3)
(146, 3)


In [41]:
tf.random.set_seed(111)

conv = tf.keras.layers.Conv2D(64, 2, strides=(2, 2))
flatten = tf.keras.layers.Reshape((-1, 1024))

In [ ]:
# TRY DATACOLLATOR or IterableDataset

In [12]:
def data_generator(file_paths_df, conv, flatten, n_frames=256):
    for _, row in file_paths_df.iterrows():
        with fs.open(row['movinet_path'], 'rb') as f:
            movinet_output = np.load(f)

        vid_embedding = conv(movinet_output)
        vid_embedding = flatten(vid_embedding)[0]
        
        padding = tf.zeros((tf.constant(256) - vid_embedding.shape[0], 
                            vid_embedding.shape[1]))
        
        attention_mask = tf.concat([tf.ones(vid_embedding.shape[0]), 
                                    tf.zeros(tf.constant(256) - vid_embedding.shape[0])], 
                                   axis=0)
        
        vid_embedding = tf.concat([vid_embedding, padding], axis=0)

        label = tokenizer(row['caption'], return_tensors="tf", 
                           padding='max_length', max_length=64).input_ids[0]
        # label = row['caption']
        
        yield {'inputs_embeds':vid_embedding, #'input_ids':None, 'training':True,
               'attention_mask':attention_mask, 'labels':label}


# Create TensorFlow Dataset for model input. 
train_dataset = tf.data.Dataset.from_generator(
    generator=lambda: data_generator(train_files[:32], conv, flatten, n_frames=256),
    output_signature=({'inputs_embeds':tf.TensorSpec(shape=(256, 1024), dtype=tf.float32),
                       'attention_mask':tf.TensorSpec(shape=(256), dtype=tf.float32), 
                       'labels':tf.TensorSpec(shape=(64), dtype=tf.int32)}
                       ))

test_dataset = tf.data.Dataset.from_generator(
    generator=lambda: data_generator(test_files, conv, flatten, n_frames=256),
    output_signature=({'inputs_embeds':tf.TensorSpec(shape=(256, 1024), dtype=tf.float32),
                       'attention_mask':tf.TensorSpec(shape=(256), dtype=tf.float32), 
                       'labels':tf.TensorSpec(shape=(64), dtype=tf.int32)}
                       ))

# # Create TensorFlow Dataset for model input. 
# train_dataset = tf.data.Dataset.from_generator(
#     functools.partial(data_generator, video_files, conv, flatten, n_frames=256),
#     output_signature=tf.TensorSpec(shape=(), dtype=tf.string) )
                       

In [13]:
# train_dataset.as_numpy_iterator()

In [14]:
# model.prepare_tf_dataset()

In [15]:
# model(test, training=True)

In [16]:
# # FIGURE OUT OPTIMIZER FOR t5
# with tf.device("/GPU:0"):

optimizer = tf.keras.optimizers.AdamW(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss)
model.fit(iter(train_dataset.batch(16)), #validation_data=test_dataset.batch(16) validation_batch_size=16,
               epochs=3, batch_size=16, verbose=1)

Epoch 1/3


2023-11-11 05:41:45.883182: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8903
2023-11-11 05:41:47.781767: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at constant_op.cc:173 : INVALID_ARGUMENT: Dimension -40 must be >= 0
2023-11-11 05:41:47.782801: W tensorflow/core/framework/op_kernel.cc:1816] UNKNOWN: InvalidArgumentError: {{function_node __wrapped__Fill_device_/job:localhost/replica:0/task:0/device:GPU:0}} Dimension -40 must be >= 0 [Op:Fill] name: 
Traceback (most recent call last):

  File "/opt/tensorflow/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 268, in __call__
    ret = func(*args)

  File "/opt/tensorflow/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/opt/tensorflow/lib/python3.10/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator

UnknownError: Graph execution error:

2 root error(s) found.
  (0) UNKNOWN:  InvalidArgumentError: {{function_node __wrapped__Fill_device_/job:localhost/replica:0/task:0/device:GPU:0}} Dimension -40 must be >= 0 [Op:Fill] name: 
Traceback (most recent call last):

  File "/opt/tensorflow/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 268, in __call__
    ret = func(*args)

  File "/opt/tensorflow/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/opt/tensorflow/lib/python3.10/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/tmp/ipykernel_24859/2854350578.py", line 9, in data_generator
    padding = tf.zeros((tf.constant(256) - vid_embedding.shape[0],

  File "/opt/tensorflow/lib/python3.10/site-packages/tensorflow/python/util/traceback_utils.py", line 153, in erro

In [ ]:
with fs.open("s3://asl-capstone/youtube-asl/test_sample/numpy_files/movinet_output/06kKvQp4SfM_1.npy",'rb') as f:
    movinet_output = np.load(f)

movinet_output.shape

In [79]:
captions = video_files[video_files.path.str.contains('06kKvQp4SfM_1.npy')].caption.to_list()

In [72]:
# tf.keras.layers.GlobalAveragePooling3D(keepdims=True)(vid_embedding)
# tf.keras.layers.Flatten()(vid_embedding)

vid_embedding = conv(movinet_output)
vid_embedding = flatten(vid_embedding)

padding = tf.zeros((vid_embedding.shape[0], 
                    tf.constant(256) - vid_embedding.shape[1], 
                    vid_embedding.shape[2]))

attention_mask = tf.concat([tf.ones(vid_embedding.shape), 
                            padding], axis=1)

vid_embedding = tf.concat([vid_embedding, padding], axis=1)

print(vid_embedding.shape)
print(attention_mask.shape)

(1, 256, 1024)
(1, 256, 1024)


In [ ]:
# # training
# inputs = tokenizer("The <extra_id_0> walks in <extra_id_1> park", return_tensors="tf").input_ids
# labels = tokenizer("<extra_id_0> cute dog <extra_id_1> the <extra_id_2>", return_tensors="tf").input_ids
# outputs = model(inputs, labels=labels)
# loss = outputs.loss
# logits = outputs.logits
# print(loss)
# print(logits)

# # inference
# inputs = tokenizer(
#    [ "studies have shown that owning a cat is good for you", "i have always wanted to have a pet"], return_tensors="pt", padding=True
# ).input_ids  # Batch size 1

# # Convert from tokens to embeddings
# encoder_outputs = encoder_model(input_ids=inputs)
# encoder_outputs['last_hidden_state'] = encoder_outputs['last_hidden_state'].detach().numpy()
# print(encoder_outputs['last_hidden_state'].shape)

# # Generate text from embeddings
# outputs = model.generate(encoder_outputs=encoder_outputs)
# # outputs = model.generate(inputs)


# print("Response: \n")
# print(tokenizer.batch_decode(outputs, skip_special_tokens=True))
# # studies have shown that owning a dog is good for you

In [80]:
print(vid_embedding.shape)
print(len(captions))

(1, 256, 1024)
1


In [81]:
# training
embeddings = tf.convert_to_tensor(vid_embedding)
mask = tf.convert_to_tensor(attention_mask)

labels = tokenizer(captions, return_tensors="tf", padding=True).input_ids
outputs = model(input_ids=None, inputs_embeds=embeddings, 
                     attention_mask=mask, labels=labels, training=True)
print(outputs.loss)
# loss = outputs.loss
# logits = outputs.logits
# print(loss)
# print(logits)

2023-11-09 22:06:51.449044: W tensorflow/core/framework/op_kernel.cc:1816] INVALID_ARGUMENT: required broadcastable shapes


InvalidArgumentError: Exception encountered when calling layer 'SelfAttention' (type TFT5Attention).

{{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} required broadcastable shapes [Op:AddV2] name: 

Call arguments received by layer 'SelfAttention' (type TFT5Attention):
  • hidden_states=tf.Tensor(shape=(1, 256, 1024), dtype=float32)
  • mask=tf.Tensor(shape=(1, 1, 256, 1024), dtype=float32)
  • key_value_states=None
  • position_bias=None
  • past_key_value=None
  • layer_head_mask=None
  • query_length=None
  • use_cache=False
  • training=True
  • output_attentions=False

In [23]:
captions[0]

'Welcome to the third round'

In [21]:
# model.generate(encoder_outputs=encoder_outputs)
tokenizer.decode(model.generate(inputs_embeds=encoder_outputs)[0], skip_special_tokens=True)

/opt/tensorflow/lib/python3.10/site-packages/transformers/generation/tf_utils.py:838: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
2023-11-08 04:39:47.301175: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55703850 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-08 04:39:47.301212: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A10G, Compute Capability 8.6
2023-11-08 04:39:47.408791: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-08 04:39:47.854060: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8903
2023-11-08 04:39:48.206243: I ./tensorflow/compiler/jit/device_compiler.h:186]

's its its it so it. so when it so it so it so it'

In [ ]:
def movinet_T5_model(video_base_model, language_base_model,
                      max_sequence_length=120,
                      hidden_size = 1024,
                      dropout=0.1,
                      learning_rate=0.00005):
    """
    Builds a translation model that accepts videos, creates video embeddings using a MoViNet model,
    and then passes the video embeddings into an encoder-decoder to generate text translation
    """

    # Set all models to trainable
    video_base_model.trainable = True
    language_base_model.trainable = True

    # Set input structure for video inputs
    video_inputs = tf.keras.layers.Input(shape=(max_sequence_length, 256, 256, 3), dtype=tf.int64, name='input_layer')
    # token_type_ids = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='attention_mask_layer')

    # Get output embeddings from video model
    movinet_out = video_base_model(video_inputs)
    vid_embedding = movinet_out[0]['block4_layer3']

    # Layer to flatten w x h x c dimensions
    vid_embedding_flatten = tf.keras.layers.Reshape((120, 8 * 8 * 168), name='flatten')(vid_embedding)

    # Hidden layer to force embeddings into correct shape for language model
    vid_embedding_flatten = tf.keras.layers.Dense(1024, activation='linear', name='hidden_layer')(vid_embedding_flatten)

    # # Pass embeddings into T5 language model
    # language_output = language_base_model.generate(encoder_outputs=encoder_outputs, attention_mask=attention_mask)

    # classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden)

    # classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])

    # classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    #                              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    #                              metrics='accuracy')



    ### END YOUR CODE

    model = tf.keras.Model(inputs=inputs, outputs=vid_embedding_flatten)

    return model